In [1]:
!hdfs dfs -ls /taxi/raw/

Found 16 items
drwxr-xr-x   - cluster supergroup          0 2025-04-12 12:46 /taxi/raw/2009
drwxr-xr-x   - cluster supergroup          0 2025-04-12 12:50 /taxi/raw/2010
drwxr-xr-x   - cluster supergroup          0 2025-04-12 12:51 /taxi/raw/2011
drwxr-xr-x   - cluster supergroup          0 2025-04-12 12:52 /taxi/raw/2012
drwxr-xr-x   - cluster supergroup          0 2025-04-12 12:54 /taxi/raw/2013
drwxr-xr-x   - cluster supergroup          0 2025-04-12 12:55 /taxi/raw/2014
drwxr-xr-x   - cluster supergroup          0 2025-04-12 12:29 /taxi/raw/2015
drwxr-xr-x   - cluster supergroup          0 2025-04-12 12:31 /taxi/raw/2016
drwxr-xr-x   - cluster supergroup          0 2025-04-12 12:32 /taxi/raw/2017
drwxr-xr-x   - cluster supergroup          0 2025-04-12 12:33 /taxi/raw/2018
drwxr-xr-x   - cluster supergroup          0 2025-04-12 12:34 /taxi/raw/2019
drwxr-xr-x   - cluster supergroup          0 2025-04-12 12:34 /taxi/raw/2020
drwxr-xr-x   - cluster supergroup          0 2025-04-12 12:35

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("PreProcessing") \
    .getOrCreate()

25/04/12 15:09:40 WARN Utils: Your hostname, bdlc-002 resolves to a loopback address: 127.0.1.1; using 10.176.129.5 instead (on interface ens192)
25/04/12 15:09:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/12 15:09:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# All Data

## Renaming Columns, Payment Types, Pick Columns, Casting

In [3]:
df_all_raw = spark.read.parquet(f"/taxi/raw/2019/yellow_tripdata_2019-01.parquet")

In [4]:
df_all_raw.show(2)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2019-01-01 00:46:40|  2019-01-01 00:53:20|            1.0|          1.5|       1.0|                 N|         151|         239|           1|        7.0|  0.5|    0.5|      1.6

In [5]:
from pyspark.sql.functions import when, expr, col

df = df_all_raw\
.withColumnRenamed("Trip_Pickup_DateTime","pickup_datetime")\
.withColumnRenamed("Trip_Dropoff_DateTime","dropoff_datetime")\
.withColumnRenamed("tpep_pickup_datetime","pickup_datetime")\
.withColumnRenamed("tpep_dropoff_datetime","dropoff_datetime")\
.withColumnRenamed("Passenger_Count","passenger_count")\
.withColumnRenamed("Trip_Distance","trip_distance")\
.withColumnRenamed("Payment_Type","payment_type")\
.withColumnRenamed("Tip_Amt","tip_amount")\
.withColumnRenamed("Total_Amt","total_amount")\
.withColumn('payment_type', \
              when(col("payment_type") == "Credit", 1)\
              .when(col("payment_type") == "CREDIT", 1)\
              .when(col("payment_type") == "CRD", 1)\
              .when(col("payment_type") == "Cre", 1)\
              .when(col("payment_type") == "CRE", 1)\
              .when(col("payment_type") == "CASH", 2)\
              .when(col("payment_type") == "Cash", 2)\
              .when(col("payment_type") == "CSH", 2)\
              .when(col("payment_type") == "CAS", 2)\
              .when(col("payment_type") == "Cas", 2)\
              .when(col("payment_type") == "No Charge", 3)\
              .when(col("payment_type") == "NO CHARGE", 3)\
              .when(col("payment_type") == "NOC", 3)\
              .when(col("payment_type") == "NO ", 3)\
              .when(col("payment_type") == "Dispute", 4)\
              .when(col("payment_type") == "DISPUTE", 4)\
              .when(col("payment_type") == "DIS", 4)\
              .when(col("payment_type") == "Dis", 4)\
              .when(col("payment_type") == "Unknown", 5)\
              .when(col("payment_type") == "UNKNOWN", 5)\
              .when(col("payment_type") == "UNK", 5)\
              .when(col("payment_type") == "Voided Trip", 6)\
              .when(col("payment_type") == "VOIDED TRIP", 6)\
              .otherwise(expr("payment_type")))\
.selectExpr(\
    "cast(pickup_datetime as timestamp)", \
    "cast(dropoff_datetime as timestamp)", \
    "cast(passenger_count as long)", \
    "trip_distance", \
    "cast(payment_type as string)", \
    "tip_amount", \
    "total_amount" \
)

In [6]:
df.show(2)

+-------------------+-------------------+---------------+-------------+------------+----------+------------+
|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|payment_type|tip_amount|total_amount|
+-------------------+-------------------+---------------+-------------+------------+----------+------------+
|2019-01-01 00:46:40|2019-01-01 00:53:20|              1|          1.5|           1|      1.65|        9.95|
|2019-01-01 00:59:47|2019-01-01 01:18:59|              1|          2.6|           1|       1.0|        16.3|
+-------------------+-------------------+---------------+-------------+------------+----------+------------+
only showing top 2 rows



## adding month/year as columns

You can use `input_file_name` to get the filename of the dataframe. Here we have the month/year available

In [7]:
from pyspark.sql.functions import input_file_name

df.withColumn("filename", input_file_name()).show(2, False)

+-------------------+-------------------+---------------+-------------+------------+----------+------------+-------------------------------------------------------------------------------------+
|pickup_datetime    |dropoff_datetime   |passenger_count|trip_distance|payment_type|tip_amount|total_amount|filename                                                                             |
+-------------------+-------------------+---------------+-------------+------------+----------+------------+-------------------------------------------------------------------------------------+
|2019-01-01 00:46:40|2019-01-01 00:53:20|1              |1.5          |1           |1.65      |9.95        |hdfs://bdlc-002.bdlc.ls.eee.intern:9000/taxi/raw/2019/yellow_tripdata_2019-01.parquet|
|2019-01-01 00:59:47|2019-01-01 01:18:59|1              |2.6          |1           |1.0       |16.3        |hdfs://bdlc-002.bdlc.ls.eee.intern:9000/taxi/raw/2019/yellow_tripdata_2019-01.parquet|
+-------------------+----

In [8]:
from pyspark.sql.functions import input_file_name, split

df.withColumn("filename", split(input_file_name(), "-")).show(2, False)

+-------------------+-------------------+---------------+-------------+------------+----------+------------+-----------------------------------------------------------------------------------------+
|pickup_datetime    |dropoff_datetime   |passenger_count|trip_distance|payment_type|tip_amount|total_amount|filename                                                                                 |
+-------------------+-------------------+---------------+-------------+------------+----------+------------+-----------------------------------------------------------------------------------------+
|2019-01-01 00:46:40|2019-01-01 00:53:20|1              |1.5          |1           |1.65      |9.95        |[hdfs://bdlc, 002.bdlc.ls.eee.intern:9000/taxi/raw/2019/yellow_tripdata_2019, 01.parquet]|
|2019-01-01 00:59:47|2019-01-01 01:18:59|1              |2.6          |1           |1.0       |16.3        |[hdfs://bdlc, 002.bdlc.ls.eee.intern:9000/taxi/raw/2019/yellow_tripdata_2019, 01.parquet]|
+----

In [9]:
df = df\
.withColumn("filename", split(input_file_name(), "-"))\
.selectExpr("*", "substring_index(element_at(filename, 2), '_', -1) as year", "substring_index(element_at(filename, -1), '.' , 1) as month")\
.drop("filename")

In [10]:
df.show(2)

+-------------------+-------------------+---------------+-------------+------------+----------+------------+----+-----+
|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|payment_type|tip_amount|total_amount|year|month|
+-------------------+-------------------+---------------+-------------+------------+----------+------------+----+-----+
|2019-01-01 00:46:40|2019-01-01 00:53:20|              1|          1.5|           1|      1.65|        9.95|2019|   01|
|2019-01-01 00:59:47|2019-01-01 01:18:59|              1|          2.6|           1|       1.0|        16.3|2019|   01|
+-------------------+-------------------+---------------+-------------+------------+----------+------------+----+-----+
only showing top 2 rows



## convert trip distance to km instead of miles

In [11]:
from pyspark.sql.functions import expr

df = df.withColumn("trip_distance", expr("trip_distance * 1.60934"))

## adding a field `trip_amount`

In [12]:
df = df.withColumn("trip_amount", expr("total_amount-tip_amount"))

## Union Data

In [13]:
from pyspark.sql.functions import when, expr, col, input_file_name, split
   
def read(year, month):
    df = spark.read.parquet(f"/taxi/raw/{year}/yellow_tripdata_{year}-{month}.parquet")\
    
    df = df\
    .withColumnRenamed("Trip_Pickup_DateTime","pickup_datetime")\
    .withColumnRenamed("Trip_Dropoff_DateTime","dropoff_datetime")\
    .withColumnRenamed("tpep_pickup_datetime","pickup_datetime")\
    .withColumnRenamed("tpep_dropoff_datetime","dropoff_datetime")\
    .withColumnRenamed("Passenger_Count","passenger_count")\
    .withColumnRenamed("Trip_Distance","trip_distance")\
    .withColumnRenamed("Payment_Type","payment_type")\
    .withColumnRenamed("Tip_Amt","tip_amount")\
    .withColumnRenamed("Total_Amt","total_amount")\
    .withColumn('payment_type', \
                  when(col("payment_type") == "Credit", 1)\
                  .when(col("payment_type") == "CREDIT", 1)\
                  .when(col("payment_type") == "CRD", 1)\
                  .when(col("payment_type") == "Cre", 1)\
                  .when(col("payment_type") == "CRE", 1)\
                  .when(col("payment_type") == "CASH", 2)\
                  .when(col("payment_type") == "Cash", 2)\
                  .when(col("payment_type") == "CSH", 2)\
                  .when(col("payment_type") == "CAS", 2)\
                  .when(col("payment_type") == "Cas", 2)\
                  .when(col("payment_type") == "No Charge", 3)\
                  .when(col("payment_type") == "NO CHARGE", 3)\
                  .when(col("payment_type") == "NOC", 3)\
                  .when(col("payment_type") == "NO ", 3)\
                  .when(col("payment_type") == "Dispute", 4)\
                  .when(col("payment_type") == "DISPUTE", 4)\
                  .when(col("payment_type") == "DIS", 4)\
                  .when(col("payment_type") == "Dis", 4)\
                  .when(col("payment_type") == "Unknown", 5)\
                  .when(col("payment_type") == "UNKNOWN", 5)\
                  .when(col("payment_type") == "UNK", 5)\
                  .when(col("payment_type") == "Voided Trip", 6)\
                  .when(col("payment_type") == "VOIDED TRIP", 6)\
                  .otherwise(expr("payment_type")))\
    .selectExpr(\
        "cast(pickup_datetime as timestamp)", \
        "cast(dropoff_datetime as timestamp)", \
        "cast(passenger_count as long)", \
        "trip_distance", \
        "cast(payment_type as string)", \
        "tip_amount", \
        "total_amount" \
    )
    

    df = df\
    .withColumn("filename", split(input_file_name(), "-"))\
    .selectExpr("*", "substring_index(element_at(filename, 2), '_', -1) as year", "substring_index(element_at(filename, -1), '.' , 1) as month")\
    .drop("filename")
    

    df = df.withColumn("trip_distance", expr("trip_distance * 1.60934"))
    df = df.withColumn("trip_amount", expr("total_amount-tip_amount"))
    return df

In [14]:
my_dfs = []
for year in ["2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023", "2024"]:
    if year == "2025":
        !echo processing {year}/01
        my_dfs.append(read(year, "01"))
        !echo processing {year}/02
        my_dfs.append(read(year, "02"))
    else:
        for month in ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]:
            !echo processing {year}/{month}
            my_dfs.append(read(year, month))

processing 2009/01
processing 2009/02
processing 2009/03
processing 2009/04
processing 2009/05
processing 2009/06
processing 2009/07
processing 2009/08
processing 2009/09
processing 2009/10
processing 2009/11
processing 2009/12
processing 2010/01
processing 2010/02
processing 2010/03
processing 2010/04
processing 2010/05
processing 2010/06
processing 2010/07
processing 2010/08
processing 2010/09
processing 2010/10
processing 2010/11
processing 2010/12
processing 2011/01
processing 2011/02
processing 2011/03
processing 2011/04
processing 2011/05
processing 2011/06
processing 2011/07
processing 2011/08
processing 2011/09
processing 2011/10
processing 2011/11
processing 2011/12
processing 2012/01
processing 2012/02
processing 2012/03
processing 2012/04
processing 2012/05
processing 2012/06
processing 2012/07
processing 2012/08
processing 2012/09
processing 2012/10
processing 2012/11
processing 2012/12
processing 2013/01
processing 2013/02
processing 2013/03
processing 2013/04
processing 2

In [15]:
from functools import reduce
from pyspark.sql import DataFrame

df = reduce(DataFrame.unionAll, my_dfs)

In [16]:
df.show(2)

25/04/12 15:10:48 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
25/04/12 15:10:48 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
25/04/12 15:10:48 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB


+-------------------+-------------------+---------------+-----------------+------------+----------+------------+----+-----+-----------+
|    pickup_datetime|   dropoff_datetime|passenger_count|    trip_distance|payment_type|tip_amount|total_amount|year|month|trip_amount|
+-------------------+-------------------+---------------+-----------------+------------+----------+------------+----+-----+-----------+
|2009-01-04 02:52:00|2009-01-04 03:02:00|              1|        4.2325642|           2|       0.0|         9.4|2009|   01|        9.4|
|2009-01-04 03:31:00|2009-01-04 03:38:00|              3|7.322496999999999|           1|       2.0|        14.6|2009|   01|       12.6|
+-------------------+-------------------+---------------+-----------------+------------+----------+------------+----+-----+-----------+
only showing top 2 rows



In [17]:
df.printSchema()

root
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- trip_amount: double (nullable = true)



In [18]:
df = df.selectExpr(\
    "cast(year as int)", \
    "cast(month as int)", \
    "pickup_datetime", \
    "dropoff_datetime", \
    "passenger_count", \
    "trip_distance", \
    "payment_type", \
    "tip_amount", \
    "trip_amount", \
    "total_amount" \
)

In [19]:
df.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- trip_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)



In [20]:
from pyspark.sql import functions as F

# Suppose you have a DataFrame called df
df.select(F.min("pickup_datetime")).show()
df.select(F.max("pickup_datetime")).show()
df.select(F.min("dropoff_datetime")).show()
df.select(F.max("dropoff_datetime")).show()


+--------------------+
|min(pickup_datetime)|
+--------------------+
| 2001-01-01 00:01:48|
+--------------------+



+--------------------+
|max(pickup_datetime)|
+--------------------+
| 2098-09-11 02:23:31|
+--------------------+



+---------------------+
|min(dropoff_datetime)|
+---------------------+
|  1900-01-01 00:00:00|
+---------------------+



[Stage 220:=================================================>(5766 + 23) / 5789]

+---------------------+
|max(dropoff_datetime)|
+---------------------+
|  2253-08-23 07:56:38|
+---------------------+



In [21]:
df.count()

1778238040

In [22]:
from pyspark.sql.functions import lit

filtered_df = df\
    .filter(col("pickup_datetime") < lit("2024-12-31")) \
    .filter(col("pickup_datetime") > lit("2009-01-01")) \
    .filter(col("dropoff_datetime") < lit("2024-12-31")) \
    .filter(col("dropoff_datetime") > lit("2009-01-01"))

In [23]:
filtered_df.count()

25/04/12 15:11:58 WARN DAGScheduler: Broadcasting large task binary with size 1426.6 KiB
                                                                                

1778147044

In [24]:
filtered_df.show()

25/04/12 15:12:28 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB
25/04/12 15:12:29 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB
25/04/12 15:12:29 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB


+----+-----+-------------------+-------------------+---------------+------------------+------------+----------+------------------+------------+
|year|month|    pickup_datetime|   dropoff_datetime|passenger_count|     trip_distance|payment_type|tip_amount|       trip_amount|total_amount|
+----+-----+-------------------+-------------------+---------------+------------------+------------+----------+------------------+------------+
|2009|    1|2009-01-04 02:52:00|2009-01-04 03:02:00|              1|         4.2325642|           2|       0.0|               9.4|         9.4|
|2009|    1|2009-01-04 03:31:00|2009-01-04 03:38:00|              3| 7.322496999999999|           1|       2.0|              12.6|        14.6|
|2009|    1|2009-01-03 15:43:00|2009-01-03 15:57:00|              5|         16.656669|           1|      4.74|23.700000000000003|       28.44|
|2009|    1|2009-01-01 20:52:58|2009-01-01 21:14:00|              1|            8.0467|           1|      3.05|15.399999999999999|      

## Write results 

In [25]:
!hdfs dfs -rm -r /taxi/raw_all.parquet/

Deleted /taxi/raw_all.parquet


In [ ]:
filtered_df.repartition(44).write.parquet(f"/taxi/raw_all.parquet")

## Stopping Spark 

In [27]:
spark.stop()